In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import tree

In [14]:
# 学習データの読み込み
train_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/train.csv")
# テストデータの読み込み
test_df=pd.read_csv("/Users/tasuke0630/making/DataScience/kaggle/Taitanic/test.csv")
# 学習データとテストデータを連結する
df=pd.concat([train_df,test_df],ignore_index=True)

#nullのCSV化
df_null=df.isnull().sum()
df_null.to_csv("null.csv")

# 元データをコピー
df2=df.copy()

In [4]:
#null_csvを見ながら,nullのデータの確認
#sns.countplot("Survived",data=df2)
#sns.countplot("Age",data=df2)
#sns.countplot("Fare",data=df2)
#sns.countplot("Cabin",data=df2)
#sns.countplot("Embarked",data=df2)
#print(df2.Cabin.mode())
#print(df2.Fare.mode())

In [6]:
# 欠損値の補完
df2.Fare=df2.Fare.fillna(8.05)
df2.Cabin=df2.Cabin.fillna("C23")
df2.Embarked=df2.Embarked.fillna("S")

Age_mdeian=df2.Age.median()
df2.Age=df2.Age.fillna(Age_mdeian)

In [7]:
df2.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [8]:
#使わないカラムを削除する
df3=df2.drop(columns=["Name","Ticket"])
df3

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0.0,3,male,22.0,1,0,7.25,C23,S
1,2,1.0,1,female,38.0,1,0,71.2833,C85,C
2,3,1.0,3,female,26.0,0,0,7.925,C23,S
3,4,1.0,1,female,35.0,1,0,53.1,C123,S
4,5,0.0,3,male,35.0,0,0,8.05,C23,S
...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,28.0,0,0,8.05,C23,S
1305,1306,NaN,1,female,39.0,0,0,108.9,C105,C
1306,1307,NaN,3,male,38.5,0,0,7.25,C23,S
1307,1308,NaN,3,male,28.0,0,0,8.05,C23,S


In [9]:
#カテゴリカル変数の数値変換
tmp_Sex=pd.get_dummies(df3["Sex"],prefix="Sex")
df3=pd.concat([df3,tmp_Sex],axis=1).drop(columns="Sex")

tmp_Cabin=pd.get_dummies(df3["Cabin"],prefix="Cabin")
df3=pd.concat([df3,tmp_Cabin],axis=1).drop(columns="Cabin")

tmp_Embarked=pd.get_dummies(df3["Embarked"],prefix="Embarked")
df3=pd.concat([df3,tmp_Embarked],axis=1).drop(columns="Embarked")
df3

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A10,...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,22.0,1,0,7.25,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1.0,1,38.0,1,0,71.2833,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,1.0,3,26.0,0,0,7.925,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1.0,1,35.0,1,0,53.1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0.0,3,35.0,0,0,8.05,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,28.0,0,0,8.05,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1305,1306,NaN,1,39.0,0,0,108.9,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1306,1307,NaN,3,38.5,0,0,7.25,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1307,1308,NaN,3,28.0,0,0,8.05,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# 学習データに分割した結果を変数trainに格納する
train=df3[~df3.Survived.isnull()]

# テストデータに分割した結果を変数trainに格納する
test=df3[df3.Survived.isnull()]

# SalePriceを削除
test=test.drop(columns=["Survived"])

# 正解をy_trainに格納する
y_train=train.Survived

# 特徴量をX_trainに格納する
X_train=train.drop(columns=["Survived"])

In [11]:
# 決定木モデルの準備(回帰)
model=tree.DecisionTreeClassifier()

# 決定木モデルの作成
model.fit(X_train,y_train)

# 作成した決定木モデルを使った予測をおこなう
y_pred=model.predict(test)

In [12]:
# 予測結果の確認をする
y_pred

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 1., 1.

In [13]:
# テストデータと予測結果の大きさを確認する
len(test),len(y_pred)

# 予測結果をテストデータに反映する
test["Survived"]=y_pred

# 提出用のデータマートを作成する
pred_df=test[["PassengerId","Survived"]].set_index("PassengerId")

# 予測結果を整数に変換する
pred_df.Survived=pred_df.Survived.astype(int)
print(pred_df.Survived.head())

# CSVの作成
pred_df.to_csv("predict1.csv",index_label=["PassengerId"])

PassengerId
892    0
893    0
894    0
895    0
896    1
Name: Survived, dtype: int64
